# TD 6 Aurelien Pouxviel - Hugging Face Transformers

In [1]:
import os
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense
from sklearn.model_selection import train_test_split

In [10]:
def load_reviews_from_directory(directory):
    reviews = []
    labels = []
    
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            with open(os.path.join(directory, filename), "r", encoding="utf-8") as file:
                review = file.read()
                reviews.append(review)
                if directory.endswith("pos"):
                    labels.append("positive")
                elif directory.endswith("neg"):
                    labels.append("negative")
    
    return reviews, labels

# Load positive and negative reviews
positive_reviews, positive_labels = load_reviews_from_directory("C:/Users/aurel/OneDrive - De Vinci/ONE DRIVE PC/A5/NLP/TD2/txt_sentoken/pos")
negative_reviews, negative_labels = load_reviews_from_directory("C:/Users/aurel/OneDrive - De Vinci/ONE DRIVE PC/A5/NLP/TD2/txt_sentoken/neg")

# Combine positive and negative reviews and labels
all_reviews = positive_reviews + negative_reviews
all_labels = positive_labels + negative_labels

# Example: print the first positive and negative reviews
print("Positive Review:")
print(positive_reviews[0])
print("Label:", positive_labels[0])

print("\nNegative Review:")
print(all_reviews[len(positive_reviews)])
print("Label:", all_labels[len(positive_labels)])


Positive Review:
films adapted from comic books have had plenty of success , whether they're about superheroes ( batman , superman , spawn ) , or geared toward kids ( casper ) or the arthouse crowd ( ghost world ) , but there's never really been a comic book like from hell before . 
for starters , it was created by alan moore ( and eddie campbell ) , who brought the medium to a whole new level in the mid '80s with a 12-part series called the watchmen . 
to say moore and campbell thoroughly researched the subject of jack the ripper would be like saying michael jackson is starting to look a little odd . 
the book ( or " graphic novel , " if you will ) is over 500 pages long and includes nearly 30 more that consist of nothing but footnotes . 
in other words , don't dismiss this film because of its source . 
if you can get past the whole comic book thing , you might find another stumbling block in from hell's directors , albert and allen hughes . 
getting the hughes brothers to direct this

## Transformers

In [5]:
from transformers import pipeline

sentiment_analyzer = pipeline("sentiment-analysis") #default = distilbert-base-uncased-finetuned-sst-2-english

max_length = 512  # À ajuster selon les besoins du modèle

# Tronque les revues pour s'adpater au model
positive_reviews2 = [review[:max_length] for review in positive_reviews]
negative_reviews2 = [review[:max_length] for review in negative_reviews]

positive_results = sentiment_analyzer(positive_reviews2)

negative_results = sentiment_analyzer(negative_reviews2)


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Résultats pour les revues positives:
Review: POSITIVE, True Label: positive
Review: POSITIVE, True Label: positive
Review: NEGATIVE, True Label: positive
Review: POSITIVE, True Label: positive
Review: NEGATIVE, True Label: positive
Review: NEGATIVE, True Label: positive
Review: POSITIVE, True Label: positive
Review: POSITIVE, True Label: positive
Review: NEGATIVE, True Label: positive
Review: NEGATIVE, True Label: positive
Review: POSITIVE, True Label: positive
Review: POSITIVE, True Label: positive
Review: POSITIVE, True Label: positive
Review: POSITIVE, True Label: positive
Review: POSITIVE, True Label: positive
Review: POSITIVE, True Label: positive
Review: POSITIVE, True Label: positive
Review: POSITIVE, True Label: positive
Review: POSITIVE, True Label: positive
Review: POSITIVE, True Label: positive
Review: POSITIVE, True Label: positive
Review: NEGATIVE, True Label: positive
Review: POSITIVE, True Label: positive
Review: POSITIVE, True Label: positive
Review: POSITIVE, True Labe

In [7]:
from sklearn.metrics import accuracy_score
positive_predictions = [1 if result['label'] == 'POSITIVE' else 0 for result in positive_results]
negative_predictions = [1 if result['label'] == 'POSITIVE' else 0 for result in negative_results]

positive_labels_binary = [1 if label == 'positive' else 0 for label in positive_labels]
negative_labels_binary = [1 if label == 'positive' else 0 for label in negative_labels]

accuracy_positive = accuracy_score(positive_labels_binary, positive_predictions)
accuracy_negative = accuracy_score(negative_labels_binary, negative_predictions)

print(f"Précision pour les revues positives: {accuracy_positive * 100:.2f}%")
print(f"Précision pour les revues négatives: {accuracy_negative * 100:.2f}%")

Précision pour les revues positives: 69.00%
Précision pour les revues négatives: 73.50%


#### ON Wp2, we had a 

Positive Reviews Accuracy: 90.60%
Negative Reviews Accuracy: 31.90%

### So our model is now hyper balanced

### try another transformer : nlptown/bert-base-multilingual-uncased-sentiment


In [12]:
sentiment_analyzer = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")
max_length = 512  # À ajuster selon les besoins du modèle

# Tronque les revues pour s'adpater au model
positive_reviews2 = [review[:max_length] for review in positive_reviews]
negative_reviews2 = [review[:max_length] for review in negative_reviews]
positive_results = sentiment_analyzer(positive_reviews2)
negative_results = sentiment_analyzer(negative_reviews2)

In [19]:
threshold = 0.5
# Convertir les probabilités en étiquettes
predicted_positive_labels = ["positive" if result['score'] >= threshold else "negative" for result in positive_results]
predicted_negative_labels = ["positive" if result['score'] >= threshold else "negative" for result in negative_results]

accuracy_positive = accuracy_score(true_positive_labels, predicted_positive_labels)
accuracy_negative = accuracy_score(true_negative_labels, predicted_negative_labels)

print(f"Accuracy on positive reviews: {accuracy_positive * 100:.2f}%")
print(f"Accuracy on negative reviews: {accuracy_negative * 100:.2f}%")

Accuracy on positive reviews: 25.30%
Accuracy on negative reviews: 71.10%


#### This model is unbalenced and do not have good performance.

### finiteautomata/bertweet-base-sentiment-analysis

In [24]:
sentiment_analyzer = pipeline("sentiment-analysis", model="finiteautomata/bertweet-base-sentiment-analysis")
max_length = 128 #this model has a small lenght

positive_reviews_processed = [review[:max_length] for review in positive_reviews2]
negative_reviews_processed = [review[:max_length] for review in negative_reviews2]

positive_results = [sentiment_analyzer(review) for review in positive_reviews_processed]
negative_results = [sentiment_analyzer(review) for review in negative_reviews_processed]

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


In [29]:
# Convert model's predicted labels to match the format of true labels
predicted_labels_positive = ["positive" if result[0]['label'] == 'POS' else "negative" for result in positive_results]
predicted_labels_negative = ["positive" if result[0]['label'] == 'POS' else "negative" for result in negative_results]

# Count correct predictions for reviews
correct_predictions_positive = sum(1 for pred, true in zip(predicted_labels_positive, true_labels_positive) if pred == true)

correct_predictions_negative = sum(1 for pred, true in zip(predicted_labels_negative, true_labels_negative) if pred == true)

accuracy_positive = correct_predictions_positive / len(true_labels_positive) * 100
accuracy_negative = correct_predictions_negative / len(true_labels_negative) * 100

print(f"Accuracy on positive reviews: {accuracy_positive:.2f}%")
print(f"Accuracy on negative reviews: {accuracy_negative:.2f}%")


Accuracy on positive reviews: 29.50%
Accuracy on negative reviews: 82.20%


#### This model is not very approriate and unbalenced

## In conclusion, i'v succed to do better than predecent work practice with the distilbert-base-uncased-finetuned-sst-2-english, with a global accuracy of 71%
## I could have used a pre trained model fined tuned on this specific dataset, but it would be useless